In [20]:
import numpy as np
import pandas as pd 
from datetime import date, datetime
import locale


In [42]:
# 1. Cargue en un Dataframe los datos de los accidentes de los últimos 3 años

df_2020 = pd.read_csv("../input/atus_anual_2020.csv", index_col = False)
df_2021 = pd.read_csv("../input/atus_anual_2021.csv", index_col = False)
df_2022 = pd.read_csv("../input/atus_anual_2022.csv", index_col = False)

# Concatena los DataFrames en uno solo
df_atus = [df_2020, df_2021, df_2022]
df_atus = pd.concat(df_atus)

# Imprime el DataFrame unidos
print(df_atus.head())


   COBERTURA  ID_ENTIDAD  ID_MUNICIPIO  ANIO  MES  ID_HORA  ID_MINUTO  ID_DIA  \
0  Municipal           1             1  2020    1        0          0       1   
1  Municipal           1             1  2020    1        0          0       1   
2  Municipal           1             1  2020    1        0         10       1   
3  Municipal           1             1  2020    1        2          0       1   
4  Municipal           1             1  2020    1        2         10       1   

   DIASEMANA                     URBANA  ... PEATMUERTO PEATHERIDO  \
0  Miercoles  Accidente en intersección  ...          0          0   
1  Miercoles  Accidente en intersección  ...          0          0   
2  Miercoles  Accidente en intersección  ...          0          0   
3  Miercoles  Accidente en intersección  ...          0          0   
4  Miercoles  Accidente en intersección  ...          0          0   

   CICLMUERTO  CICLHERIDO  OTROMUERTO  OTROHERIDO  NEMUERTO  NEHERIDO  \
0           0      

In [43]:
# 2. Agregue una columna tipo date que englobe las columnas correspondientes a Año, Mes, Dia, Hora y Minuto.

df_atus['FECHA'] = df_atus.apply(lambda row: f"{row['ANIO']}-{row['MES']}-{row['ID_DIA']} {row['ID_HORA']}:{row['ID_MINUTO']}", axis=1)

df_atus['FECHA'] = pd.to_datetime(df_atus['FECHA'], format='%Y-%m-%d %H:%M')

df_atus = df_atus.drop(['ANIO', 'MES', 'ID_DIA', 'ID_HORA', 'ID_MINUTO','DIASEMANA'], axis=1)


print(df_atus.head())



   COBERTURA  ID_ENTIDAD  ID_MUNICIPIO                     URBANA  \
0  Municipal           1             1  Accidente en intersección   
1  Municipal           1             1  Accidente en intersección   
2  Municipal           1             1  Accidente en intersección   
3  Municipal           1             1  Accidente en intersección   
4  Municipal           1             1  Accidente en intersección   

                    SUBURBANA                         TIPACCID  AUTOMOVIL  \
0  Sin accidente en esta zona  Colisión con vehículo automotor          1   
1  Sin accidente en esta zona         Colisión con objeto fijo          1   
2  Sin accidente en esta zona         Colisión con motocicleta          1   
3  Sin accidente en esta zona  Colisión con vehículo automotor          2   
4  Sin accidente en esta zona  Colisión con vehículo automotor          2   

   CAMPASAJ  MICROBUS  PASCAMION  ...  PEATHERIDO  CICLMUERTO  CICLHERIDO  \
0         1         0          0  ...        

In [46]:
# 3. Agregue una columna para Área cuyos valores serán Urbana o Suburbana dependiendo del área donde ocurrió el accidente.
# 4. Agregue una columna Zona cuyos valores serán la zona donde ocurrió el accidente.


df_atus['ZONA'] = df_atus.apply(lambda row: row['URBANA'] if row['URBANA'] != 'Sin accidente en esta zona' else (row['SUBURBANA'] 
if row['SUBURBANA'] != 'Sin accidente en esta zona' else None), axis=1)

df_atus['URBANA'] = (df_atus['URBANA']=='Sin accidente en esta zona').astype(int)
print(df_atus.groupby("URBANA"))

df_atus['URBANA'] = df_atus['URBANA'].apply(lambda x: 'Suburbana' if x == 1 else 'Urbana')

# 5. Elimine las columnas URBANA y SUBURBANA

df_atus = df_atus.drop(['SUBURBANA'], axis=1)
df_atus = df_atus.rename(columns = {'URBANA' : 'AREA'})


In [47]:
print(df_atus.head(120))

     COBERTURA  ID_ENTIDAD  ID_MUNICIPIO       AREA  \
0    Municipal           1             1     Urbana   
1    Municipal           1             1     Urbana   
2    Municipal           1             1     Urbana   
3    Municipal           1             1     Urbana   
4    Municipal           1             1     Urbana   
..         ...         ...           ...        ...   
115  Municipal           1             1     Urbana   
116  Municipal           1             1     Urbana   
117  Municipal           1             1     Urbana   
118  Municipal           1             1     Urbana   
119  Municipal           1             1  Suburbana   

                            TIPACCID  AUTOMOVIL  CAMPASAJ  MICROBUS  \
0    Colisión con vehículo automotor          1         1         0   
1           Colisión con objeto fijo          1         0         0   
2           Colisión con motocicleta          1         0         0   
3    Colisión con vehículo automotor          2        

In [48]:
# 7. Resuma los accidentes del último año por tipo y que porcentaje representan

TipoAccidente = df_2022['TIPACCID'].value_counts().reset_index()
TipoAccidente.columns = ['Tipo de Accidente', 'Count']

TotalAccidentes = TipoAccidente['Count'].sum()
TipoAccidente['Porcentaje sobre el Total'] = (TipoAccidente['Count'] / TotalAccidentes) * 100
print(TipoAccidente)



                        Tipo de Accidente   Count  Porcentaje sobre el Total
0         Colisión con vehículo automotor  229940                  58.552112
1                Colisión con motocicleta   53629                  13.656133
2                Colisión con objeto fijo   47033                  11.976522
3                        Certificado cero   15479                   3.941585
4   Colisión con peatón (atropellamiento)   11066                   2.817855
5                               Volcadura   10911                   2.778386
6                       Salida del camino   10857                   2.764635
7                                    Otro    6325                   1.610603
8                   Colisión con ciclista    4048                   1.030786
9                       Caída de pasajero    1784                   0.454279
10                    Colisión con animal    1022                   0.260243
11                               Incendio     348                   0.088615

In [31]:
# 8. Obtenga la cantidad de muertos y lesionados de los últimos 3 años

Muertos = pd.DataFrame()

Todos_muertos = df_atus[['CONDMUERTO','PASAMUERTO','PEATMUERTO', 'CICLMUERTO', 'OTROMUERTO', 'NEMUERTO', 'CONDHERIDO','PASAHERIDO', 'PEATHERIDO', 'CICLHERIDO', 'OTROHERIDO', 'NEHERIDO']].sum()

Muertos['total'] = Todos_muertos

Muertos['categorias'] = df_atus[['CONDMUERTO','PASAMUERTO','PEATMUERTO', 'CICLMUERTO', 'OTROMUERTO', 'NEMUERTO', 'CONDHERIDO','PASAHERIDO', 'PEATHERIDO', 'CICLHERIDO', 'OTROHERIDO', 'NEHERIDO']].columns

Muertos['porcentaje sobre total'] = (Muertos['total'] / Muertos['total'].sum()) * 100


print(Muertos)



             total  categorias  porcentaje sobre total
CONDMUERTO    7050  CONDMUERTO                2.718754
PASAMUERTO    3088  PASAMUERTO                1.190853
PEATMUERTO    2689  PEATMUERTO                1.036983
CICLMUERTO     433  CICLMUERTO                0.166982
OTROMUERTO     148  OTROMUERTO                0.057075
NEMUERTO         0    NEMUERTO                0.000000
CONDHERIDO  127628  CONDHERIDO               49.218310
PASAHERIDO   78588  PASAHERIDO               30.306583
PEATHERIDO   31949  PEATHERIDO               12.320774
CICLHERIDO    6787  CICLHERIDO                2.617331
OTROHERIDO     950  OTROHERIDO                0.366357
NEHERIDO         0    NEHERIDO                0.000000


In [49]:
# 9. ¿Cómo se compara el número de accidentes de cada año contra el anterior? ¿En qué porcentaje varió?

df_atus['FECHA'] = pd.to_datetime(df_atus['FECHA'])

df_atus['AÑO'] = df_atus['FECHA'].dt.year

total_accidentes_por_año = df_atus['AÑO'].value_counts().reset_index()
total_accidentes_por_año.columns = ['AÑO', 'Total_Accidentes']

total_accidentes_por_año['Porcentaje'] = (total_accidentes_por_año['Total_Accidentes'] / total_accidentes_por_año['Total_Accidentes'].sum()) * 100

df_atus = df_atus.drop('AÑO', axis = 1)
print(total_accidentes_por_año)


    AÑO  Total_Accidentes  Porcentaje
0  2022            392710   36.802612
1  2021            356315   33.391874
2  2020            318046   29.805514
